<a href="https://colab.research.google.com/github/dzaras/review_classification/blob/main/Review_Classification_based_on_lead_actors_gender_8_18_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk
!pip install numpy 
!pip install pandas
!pip install gensim
!pip install sklearn
!pip install ktrain

     |████████████████████████████████| 25.3 MB 100 kB/s 
     |████████████████████████████████| 6.8 MB 27.8 MB/s 
     |████████████████████████████████| 981 kB 56.3 MB/s 
     |████████████████████████████████| 263 kB 58.9 MB/s 
     |████████████████████████████████| 1.9 MB 26.6 MB/s 
     |████████████████████████████████| 1.2 MB 44.8 MB/s 
     |████████████████████████████████| 468 kB 66.7 MB/s 
     |████████████████████████████████| 895 kB 69.8 MB/s 
     |████████████████████████████████| 3.3 MB 31.6 MB/s 
  Created wheel for ktrain: filename=ktrain-0.27.3-py3-none-any.whl size=25283131 sha256=521f904a4d7a77a7772607d20137f89ba51d793458ec29ce542f25d517ac90e2
  Stored in directory: /root/.cache/pip/wheels/3c/ef/e1/7da805d8a5944e8a3ac0553831d832c00b1800b848939849cb
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=9c814f41063f2c1f99abdc5aecb424b19d25dcfb90f1b877026e29a0246fec64
  Stored in directory: /root/.cache/pip/wheels/f5/ac/f1/4e13d7aff

# Metrics for Text Classification

In this section, I import the built-in functions for measuring text classification accuracy, precision, recall, and f1 score. After I input a list of predicted lables and the ground truth, the built-in functions will return the calculated results. 

## Accuracy: 

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
y_pred = [0, 2, 1, 3]
y_true = [0, 1, 2, 3]

In [ ]:
accuracy_score(y_true, y_pred)

0.5

## Precission, Recall and F1

Note: Macro vs Micro:

Macro: Calculate metrics for each label, and find their unweighted mean.

Micro: Calculate metrics globally by counting the total true positives, false negatives and false positives.

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
y_true = [0, 1, 2, 0, 1, 2]
y_pred = [0, 2, 1, 0, 0, 1]

In [ ]:
precision_score(y_true, y_pred, average='micro')

0.3333333333333333

In [ ]:
recall_score(y_true, y_pred, average='micro')

0.3333333333333333

In [ ]:
f1_score(y_true, y_pred, average='micro')

0.3333333333333333

# Text Classification 

In this section I go through the process of performing text classification with Keras. First, I extract the text representation with sklearn and then utilize the built-in model in sklearn to learn the classification model.

In this case, I am using a dataset of 2,368 amateur film reviews I collected from imdb.com that were posted in 2008. This dataset has labels for the gender (male, female) of the leading actor of the film that the review is about. 

In [ ]:
import pandas as pd

# Here I use the imdb amateur film review dataset with labels about leading actors' gender (male, female). # This version of the dataset contains 2,368 review posts. 
# This is available as https://raw.githubusercontent.com/dzaras/review_classification/main/newspaper_data_3_labels.csv

raw_data = pd.read_csv('https://raw.githubusercontent.com/dzaras/review_classification/main/amateur_reviews_08_actor_labels.csv' , encoding='latin-1')
#raw_data = pd.read_csv('https://raw.githubusercontent.com/dzaras/review_classification/main/newspaper_data_only_neg-pos.csv' , encoding='latin-1')

print(raw_data)

                                film.title  ...                                            content
0       What Happens in Vegas               ...  The story behind this Romantic Comedy is that ...
1                 Untraceable               ...  This story of a crazed killer using hits on a ...
2                   Get Smart               ...  This film is about an incompetent agent who is...
3                   Good Dick               ...  I always check the spoiler box, just in case.T...
4          Frygtelig lykkelig               ...  Nicely done. I am glad I picked this one out. ...
...                                    ...  ...                                                ...
2363             The Bank Job               ...  Having been weaned on The Italian Job, Thomas ...
2364             The Bank Job               ...  I liked the film to start with - bit cheeky, t...
2365     La siciliana ribelle               ...  While this film adds one of the faces that cou...
2366  The 

In [ ]:
raw_data.dtypes

film.title      object
author          object
target           int64
target_name     object
genre           object
year             int64
rating         float64
content         object
dtype: object

Changing the column 'Content' from a Pandas 'object' dtype to 'string' 

In [ ]:
raw_data['content'] = raw_data['content'].astype('string')

In [ ]:
raw_data.dtypes

film.title      object
author          object
target           int64
target_name     object
genre           object
year             int64
rating         float64
content         string
dtype: object

In [ ]:
raw_data

,film.title,author,target,target_name,genre,year,rating,content
0,What Happens in Vegas,MovieBuff26,1,female,comedy,2010,8.0,The story behind this Romantic Comedy is that ...
1,Untraceable,tatz32000,1,female,crime,2008,7.0,This story of a crazed killer using hits on a ...
2,Get Smart,Gordon-11,0,male,NaN,2008,NaN,This film is about an incompetent agent who is...
3,Good Dick,wingedheartart,1,female,misc,2009,6.0,"I always check the spoiler box, just in case.T..."
4,Frygtelig lykkelig,chapsmack,0,male,crime,2009,NaN,Nicely done. I am glad I picked this one out. ...
...,...,...,...,...,...,...,...,...
2363,The Bank Job,jemps918,0,male,crime,2008,5.0,"Having been weaned on The Italian Job, Thomas ..."
2364,The Bank Job,prbt,0,male,crime,2008,6.0,"I liked the film to start with - bit cheeky, t..."
2365,La siciliana ribelle,ricardodiazsoto,1,female,crime,2010,NaN,While this film adds one of the faces that cou...
2366,The Secret Life of Bees,johnstonjames,1,female,drama,2010,8.0,Wow. This was a really good movie. The only re...


In [ ]:
# Read the text for classification
text = []
#for i in range(0, len(raw_data['content'])):
for i in range(0, 1900):      # first 1000 observations for training; we could have used more or all of them
  text.append(raw_data['content'][i])

In [ ]:
# Read the labels 
labels = []
#for i in range(0, len(raw_data['target'])):
for i in range(0, 1900):      # first 1000 observations for training; we could have used more or all of them
  labels.append(raw_data['target'][i])

In [ ]:
target_name = ['female','male']

## Text Classification with Keras

In [ ]:
import ktrain
from ktrain import text as ktext
from sklearn.model_selection import train_test_split

### Forming the train and test sets

In [ ]:
# We first split the original data into train and test set
X_train, X_test, y_train, y_test = train_test_split(text, labels, random_state = 0)     #random state is like random seed

### Preprocess Data

Notice that we use distilbert in the next chunk of code where we pre-process the texts. We'd have to use the same mode (distilbert or roBERta or gpt) in the training of the model as we do in the pre-process stage.
If we wanted to use a different kind of model (such as roBERTa instead of distilbert) we'd change the preprocess_mode argument in the following chunk of code. Of course, we'd have to check the documentation of the function to make sure the mode we want to use is supported. 

In [ ]:
trn, val, preproc = ktext.texts_from_array(x_train=X_train, y_train=y_train,
                                          x_test=X_test, y_test=y_test,
                                          class_names = target_name,
                                          preprocess_mode='distilbert',
                                          maxlen=128)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

preprocessing train...
language: en
train sequence lengths:
	mean : 331
	95percentile : 833
	99percentile : 977


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 320
	95percentile : 806
	99percentile : 969


task: text classification


### Building a Model and a Learner







In [ ]:
model = ktext.text_classifier('distilbert', train_data=trn, preproc=preproc)

Is Multi-Label? False
maxlen is 128


Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

done.


Batch size is hyperparameters -> how many datapoints we can analyze simultaneously and it depends on our memory size.

In [ ]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=16)

### Train the Model

Epoch is similar to 'passes' we saw in the topic modelling section (sort of like bootstrapping for nlp)

In [ ]:
learner.fit_onecycle(3e-5, 4)



begin training using onecycle policy with max lr of 3e-05...
Epoch 1/4
90/90 [==============================] - 932s 10s/step - loss: 0.6577 - accuracy: 0.6063 - val_loss: 0.5066 - val_accuracy: 0.7747
Epoch 2/4
90/90 [==============================] - 914s 10s/step - loss: 0.4139 - accuracy: 0.8154 - val_loss: 0.3404 - val_accuracy: 0.8463
Epoch 3/4
90/90 [==============================] - 915s 10s/step - loss: 0.1854 - accuracy: 0.9375 - val_loss: 0.3333 - val_accuracy: 0.8611
Epoch 4/4
90/90 [==============================] - 917s 10s/step - loss: 0.0732 - accuracy: 0.9839 - val_loss: 0.3872 - val_accuracy: 0.8611


### Predict on New Data

In [ ]:
p = ktrain.get_predictor(model, preproc)
# save Predictor (i.e., model and Preprocessor instance) after partially training
predictor = ktrain.get_predictor(model, preproc).save('/tmp/my_predictor')



In [ ]:
# reload Predictor and extract model
model = ktrain.load_predictor('/tmp/my_predictor').model

In [ ]:
# re-instantiate Learner and continue training
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=8)
learner.fit_onecycle(2e-5, 1)